In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import math
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score,
    mean_squared_log_error,
)
from rich.console import Console
from rich.table import Table
import datetime

In [5]:
df = pd.read_csv('/Users/victoroliveira/Desktop/vonix-py-statistic/src/vonixstatisc/data/pandas_data/data_fluency_no_auto_dialer.csv')
df_march = df.loc[df['month']== 'March']
df_march = df_march.loc[(df_march['handling_time'] > 10)]
df_march_queue = df_march.loc[(df_march['queue_id']) == 'mgmsales']
df_march_queue_1140 = df_march_queue.loc[(df_march['agent_id']) == 1600]

In [6]:
df_march_queue_1140

,agent_id,queue_id,direction,locality_id,call_type_id,hold_secs,ring_secs,initial_position,trunking_id,carrier_id,week_day,day,month,hour,minute,timestamp,handling_time
11008,1600,mgmsales,OUT,84.0,5,0,7,0,1,3.0,3,1,March,19,3,1677697673,19
11387,1600,mgmsales,OUT,62.0,5,0,39,0,1,1.0,3,1,March,20,3,1677701130,13
11565,1600,mgmsales,OUT,62.0,5,0,12,0,1,67.0,3,1,March,20,3,1677702146,23
11653,1600,mgmsales,OUT,11.0,5,0,9,0,1,67.0,3,1,March,20,3,1677703018,25
11668,1600,mgmsales,OUT,84.0,5,0,13,0,1,3.0,3,1,March,20,3,1677703126,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75890,1600,mgmsales,OUT,35.0,5,0,19,0,1,5.0,5,31,March,22,3,1680301149,73
75955,1600,mgmsales,OUT,92.0,5,0,19,0,1,5.0,5,31,March,22,3,1680302018,72
75970,1600,mgmsales,OUT,44.0,4,0,43,0,1,1.0,5,31,March,22,3,1680302223,17
76004,1600,mgmsales,OUT,61.0,5,0,20,0,1,5.0,5,31,March,22,3,1680302712,15


In [7]:
dict_march_queue_1140  = df_march_queue_1140.set_index('timestamp').to_dict()['handling_time']
dict_march_queue = df_march_queue.set_index('timestamp').to_dict()['handling_time']

In [8]:
from datetime import datetime
from time import gmtime, strftime
import time
def transform_dict(data: dict, period: int) -> dict:
    """method that receives dict from database query
    {start_at(timestamp): talk_secs(seconds), ...} and create
    another dict based on the period (15, 30, 1hr) chosen
    {last_timestamp: [12,24,56,12, 90, 100], last_timestamp - period:[195,235,201,300], ... interval of three minutes for standard model}
    For standard model the data from the last period( 15 min, 30 min, 1h) is used to construct forecast for the next period so the return_dict begins with the last timestamp to the first
    """
    seconds = period * 60
    
    return_dict = {}

    keys_array = list(data.keys())
    beginning = keys_array[0]
    limit= beginning + seconds
    end = keys_array[-1]
    arr = []
    break_v = 1
    while beginning < end:
        if break_v == beginning:
            break
        break_v = beginning
        for key, value in data.items():
            if key <= limit:
                arr.append(value)
                continue
        
            return_dict[f"{beginning}"] = arr
            arr = []
            arr.append(value)
            beginning = key
            limit = beginning + seconds
            

    return return_dict
#datetime.fromtimestamp(end).strftime('%d/%m %H:%M')

In [9]:
def z_clean_outliers(time_array: list) -> list:
    """method to clean outliers of dict
    { 90: [12,24,56,12], 180: [165,135,12,9], ... interval of three minutes}
    for standard model based on z-score method"""

    threshold = 2
    arr = []
    mean = np.mean(time_array)
    std = np.std(time_array)
    for h_time in time_array:
        if h_time - mean == 0:
           arr.append(h_time)
           continue
        
        z_score = (h_time - mean) / std
        #print({'z_score':z_score, 'h_time':h_time})
        if np.abs(z_score) < threshold:
           arr.append(h_time)  
            
    return arr

In [10]:
def transform_array(array: list, interval: list, min_time: list, max_time: list):
    """Used inside StandardModel : loops using transform_dict to create dicts for the standard model
     - receives array [talk_secs from a chosen 15min, 30min 1hr period] ex : [194,30303,21,1,0,3,1231,6547,8869]
     - for standard model, interval = 180 secs, min_time = 8 secs, max_time = 5400 secs
     - filters data between min and max time [194,503,21,1231]
    {180: [21,194], 360 [303], ... interval of three minutes}"""

    limit = interval
    arr = []
    beginning = 0
    return_dict = {}

    sort_data = [
        time for time in np.sort(array) if time >= min_time and time <= max_time
    ]
    last_time = sort_data[-1]
    for time in sort_data:      
        #print({'time':time, 'limit':limit})
        if time <= limit:
            arr.append(time)
            if time == last_time:
           
                arr_clean = z_clean_outliers(arr)
                return_dict[limit] = arr
              #  print( return_dict)
            continue

        arr_clean = z_clean_outliers(arr)
        return_dict[limit] = arr
        #print({'return_dict':return_dict})
        arr = []
        while time > limit:
            beginning += interval
            limit += interval
            #print(limit)
        arr.append(time)
       # print(arr)
        if time == last_time:
        #    print(arr)
            arr_clean = z_clean_outliers(arr)
            return_dict[limit] = arr_clean

    return return_dict



In [47]:
dict_march_queue_1140_tr= transform_dict(dict_march_queue_1140, 90)
dict_march_queue_tr=  transform_dict(dict_march_queue, 90)
dict_queue = {i: transform_array(dict_march_queue_tr[i], 180, 10, 5400) for i in dict_march_queue_tr.keys()}
dict_1140 = {i: transform_array(dict_march_queue_1140_tr[i], 180, 10, 5400) for i in dict_march_queue_1140_tr.keys()}
#dict_queue = {i: transform_array(dict_march_queue_tr[i], 180, 10, 5400) for i in dict_march_queue_tr.keys()}
#z_clear only for intervals
#dict_march_queue_1140_tr_clean = {i:z_clean_outliers(dict_march_queue_1140_tr[i]) for i in dict_march_queue_1140_tr.keys() }
#dict_march_queue_tr_clean = {i:z_clean_outliers(dict_march_queue_tr[i]) for i in dict_march_queue_tr.keys() }

#dict_1140

/opt/homebrew/anaconda3/envs/vonix-statistic/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/anaconda3/envs/vonix-statistic/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/anaconda3/envs/vonix-statistic/lib/python3.11/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/anaconda3/envs/vonix-statistic/lib/python3.11/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/homebrew/anaconda3/envs/vonix-statistic/lib/python3.11/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret =

In [48]:
df_1140 = pd.DataFrame()
df_1140['timestamp'] = dict_1140.keys()
df_1140['mean'] = dict_1140.values()
df_1140['day'] = [datetime.fromtimestamp(int(i)).strftime('%d') for i in dict_1140.keys()]
df_1140['hour'] = [datetime.fromtimestamp(int(i)).strftime('%H') for i in dict_1140.keys()]
df_1140['minute'] = [datetime.fromtimestamp(int(i)).strftime('%M') for i in dict_1140.keys()]
df_1140['week_day'] = [datetime.fromtimestamp(int(i)).strftime('%A') for i in dict_1140.keys()]
#pd.set_option('display.max_rows', None)
df_1140

,timestamp,mean,day,hour,minute,week_day
0,1677697673,"{180: [13, 19, 23, 25]}",01,15,07,Wednesday
1,1677703126,"{180: [11, 22, 109, 162], 1080: [977]}",01,16,38,Wednesday
2,1677709819,"{180: [17, 18]}",01,18,30,Wednesday
3,1677763861,"{180: [13, 14, 15, 24, 70], 1440: [1325]}",02,09,31,Thursday
4,1677770097,"{180: [21, 28], 540: [513]}",02,11,14,Thursday
...,...,...,...,...,...,...
113,1680212992,"{180: [15, 18, 60], 360: [222]}",30,17,49,Thursday
114,1680275773,"{180: [14, 14, 17, 18, 18, 18, 18, 18, 19, 19,...",31,11,16,Friday
115,1680281863,"{180: [11, 12, 13, 13, 14, 14, 16, 18, 29, 53,...",31,12,57,Friday
116,1680287776,"{180: [12, 17, 38, 83, 90, 103], 540: [451]}",31,14,36,Friday


In [49]:
df_queue = pd.DataFrame()
df_queue['timestamp'] = dict_queue.keys()
df_queue['mean'] =dict_queue.values()
df_queue['day'] = [datetime.fromtimestamp(int(i)).strftime('%d') for i in dict_queue.keys()]
df_queue['hour'] = [datetime.fromtimestamp(int(i)).strftime('%H') for i in dict_queue.keys()]
df_queue['minute'] = [datetime.fromtimestamp(int(i)).strftime('%M') for i in dict_queue.keys()]
df_queue['week_day'] = [datetime.fromtimestamp(int(i)).strftime('%A') for i in dict_queue.keys()]
df_queue

,timestamp,mean,day,hour,minute,week_day
0,1677691831,"{180: [12, 16, 17, 18, 59, 83]}",01,13,30,Wednesday
1,1677697562,"{180: [11, 12, 13, 13, 13, 14, 16, 16, 17, 19,...",01,15,06,Wednesday
2,1677703018,"{180: [11, 11, 14, 18, 20, 22, 25, 27, 28, 30,...",01,16,36,Wednesday
3,1677708461,"{180: [14, 16, 17, 18, 43, 74, 87]}",01,18,07,Wednesday
4,1677763308,"{180: [13, 13, 14, 15, 24, 29, 42, 44, 45, 47,...",02,09,21,Thursday
...,...,...,...,...,...,...
157,1680271790,"{180: [11, 11, 13, 14, 14, 15, 15, 15, 16, 18,...",31,10,09,Friday
158,1680277443,"{180: [13, 13, 13, 14, 14, 14, 17, 17, 18, 18,...",31,11,44,Friday
159,1680283840,"{180: [11, 11, 11, 11, 11, 12, 12, 12, 13, 13,...",31,13,30,Friday
160,1680289432,"{180: [12, 14, 14, 14, 14, 14, 15, 16, 17, 17,...",31,15,03,Friday


In [50]:
key = list(dict_march_queue_tr.keys())
value = list(dict_march_queue_tr.values())
queue = {'queue':value[13]}

key_2 = list(dict_march_queue_1140_tr.keys())
value_2 = list(dict_march_queue_1140_tr.values())

agent = {'1140':value_2[13]}

y_true = value_2[14]


In [51]:
class StandardModel:

    """Standard model used currently for forecasting of average handling time"""

    def __init__(self, data_train: list[int], data_predict: list[int]) -> None:
        self.__data = data_train
        self.__model = self.__construct_model()
        self.predict_results = self.predict(data_predict)
        self.data_transform = transform_array(self.__data, 180, 8, 5400)

    def __construct_model(self) -> dict:
        data = transform_array(self.__data, 180, 8, 5400)
        dict_model = {i: round(np.mean(data[i])) for i in data}
        return dict_model

    def calculate_rsd(self):
        data = transform_array(self.__data, 180, 8, 5400)
        dict_rsd = {i: round(np.std(data[i]) / np.mean(data[i]), 3) for i in data}
        return dict_rsd

    def model(self):
        return self.__model

    def count_samples(self):
        data = transform_array(self.__data, 180, 8, 5400)
        dict_model = {i: len(data[i]) for i in data}
        return dict_model

    def predict(self, data_to_predict) -> dict:
        y_predict = []
        y_true = []
        for time in data_to_predict:
            last_item = list(self.__model.keys())[-1]
            for three_minute_interval, mean in self.__model.items():
                if time <= three_minute_interval:
                    y_predict.append(mean)
                    y_true.append(time)
                    break
            
        return {"y_predict": y_predict, "y_true": y_true}


In [52]:
def compareStandardModel(data: dict[str, list]) -> dict:
    """Generates Standard Model dataframe for comparison of differents agents and possibly queue"""

    rsd_dict = {}
    agents = []
    rsd = []
    period = []
    mean = []
    number_of_samples = []
    for agent, array_data_train in data.items():
        model = StandardModel(data_train=array_data_train, data_predict=[])
        model_count = model.count_samples()
        model_rsd = model.calculate_rsd()
        model_mean = model.model()

        agents.extend([str(agent)] * len(model_mean.values()))
        rsd.extend(list(model_rsd.values()))
        period.extend(list(model_rsd.keys()))
        mean.extend(list(model_mean.values()))
        number_of_samples.extend(list(model_count.values()))

    rsd_dict["agents"] = agents
    rsd_dict["rsd"] = rsd
    rsd_dict["period"] = period
    rsd_dict["mean"] = mean
    rsd_dict["number_of_samples"] = number_of_samples
    return pd.DataFrame(rsd_dict)


In [53]:
queue_model = compareStandardModel(queue)
agent_model = compareStandardModel(agent)

In [54]:
agent_model

,agents,rsd,period,mean,number_of_samples
0,1140,1.061,180,25,11


In [55]:
queue_model

,agents,rsd,period,mean,number_of_samples
0,queue,1.068,180,38,13
1,queue,0.000,720,550,1
2,queue,0.000,1080,969,1


In [56]:

def compareStandardModelMetrics(agent, agent_data_train: list, queue_data_train: list, data_to_predict: list) -> dict:
    """Generates Standard Models of an agent and the correponding queue for comparison of the metrics generated for a given period of time"""
    rsd_dict = {}

    agent_data_model = StandardModel(
        data_train=agent_data_train, data_predict=data_to_predict
    ).predict_results
    print(agent_data_model)
    
    
    queue_data_model = StandardModel(
        data_train=queue_data_train, data_predict=data_to_predict
    ).predict_results
    print(queue_data_model)
    agent_metrics = MetricFunctions(
        agent, agent_data_model["y_predict"], agent_data_model["y_true"], 1
    )
    queue_metrics = MetricFunctions(
        "queue", queue_data_model["y_predict"], queue_data_model["y_true"], 1
    )

    agent_metrics.print_table()
    queue_metrics.print_table()

    return pd.DataFrame(rsd_dict)

class MetricFunctions:

    """All metrics that can be used to compare the fitnesse for different models or between models"""

    def __init__(self, agent, y_predict, y_true, total_features) -> None:
        self.mae = round(mean_absolute_error(y_true, y_predict), 3)
        self.rmse = round(mean_squared_error(y_true, y_predict, squared=True), 3)
        self.rmsle = round(mean_squared_log_error(y_true, y_predict), 3)
        self.mape = round(mean_absolute_percentage_error(y_true, y_predict), 3)
        self.r2_score = round(r2_score(y_true, y_predict), 4)
        self.r2_score_adjusted = round(
            1
            - ((1 - r2_score(y_true, y_predict)) * (len(y_predict) - 1))
            / (len(y_true) - total_features - 1),
            4,
        )
        self.agent = agent

    def return_metrics(self):
        """returning metrics results for dataframe"""

        return_dict = {
            "agent": self.agent,
            "mean_absolute_error": self.mae,
            "root_mean_squared_error": self.rmse,
            "root_mean_squared_log_error": self.rmsle,
            "mean_absolute_percentage_error": self.mape,
            "r2_score": self.r2_score,
            "r2_score_adj": self.r2_score_adjusted,
        }

        return pd.DataFrame(return_dict)

    def print_table(self):
        """Method for pretty logging of models metrics"""
        console = Console()

        table = Table(show_header=True, header_style="bold")
        table.add_column("agent_id", justify="center")
        table.add_column("mean_absolute_error", justify="center")
        table.add_column("root_mean_squared_error", justify="center")
        table.add_column("root_mean_squared_log_error", justify="center")
        table.add_column("mean_absolute_percentage_error", justify="center")
        table.add_column("r2_score", justify="center")
        table.add_column("r2_score_adj", justify="center")

        table.add_row(
            str(self.agent),
            str(self.mae),
            str(self.rmse),
            str(self.rmsle),
            str(self.mape),
            str(self.r2_score),
            str(self.r2_score_adjusted),
        )

        console.print(table)


In [57]:
y_true

[18, 64, 11, 16, 17, 41, 18, 30, 32, 81, 31]

In [58]:
data_train = list(agent.values()).pop()
queue_train = list(queue.values()).pop()
compareStandardModelMetrics('1140', data_train,queue_train ,y_true)

{'y_predict': [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25], 'y_true': [18, 64, 11, 16, 17, 41, 18, 30, 32, 81, 31]}
{'y_predict': [38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38], 'y_true': [18, 64, 11, 16, 17, 41, 18, 30, 32, 81, 31]}


┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ agent_id ┃ mean_absolute_e… ┃ root_mean_squar… ┃ root_mean_square… ┃ mean_absolute_p… ┃ r2_score ┃ r2_score_adj ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   1140   │      15.818      │     496.545      │       0.33        │      0.487       │ -0.1331  │    -0.259    │
└──────────┴──────────────────┴──────────────────┴───────────────────┴──────────────────┴──────────┴──────────────┘

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ agent_id ┃ mean_absolute_e… ┃ root_mean_squar… ┃ root_mean_square… ┃ mean_absolute_p… ┃ r2_score ┃ r2_score_adj ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│  queue   │      18.455      │      467.0       │       0.423       │      0.816       │ -0.0656  │   -0.1841    │
└──────────┴──────────────────┴──────────────────┴───────────────────┴──────────────────┴──────────┴──────────────┘

""
